# Redes Neuronales Artificales


### Instalar Theano
pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

### Instalar Tensorflow y Keras
conda install -c conda-forge keras

## Parte 1 - Pre procesado de datos

### Cómo importar las librerías


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


### Importar el data set


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset = pd.read_csv('https://raw.githubusercontent.com/joanby/deeplearning-az/master/datasets/Part%201%20-%20Artificial%20Neural%20Networks%20(ANN)/Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
dataset.iloc[0,:].values

array([1, 15634602, 'Hargrave', 619, 'France', 'Female', 42, 2, 0.0, 1, 1,
       1, 101348.88, 1], dtype=object)

In [6]:
X[0,:]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)


### Codificar datos categóricos


In [7]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])


In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("Churn_Modelling",        # Un nombre de la transformación
         OneHotEncoder(categories='auto'), # La clase a la que transformar
         [1]            # Las columnas a transformar.
         )
    ], remainder='passthrough'
)

X = transformer.fit_transform(X)
X = X[:, 1:]


### Dividir el data set en conjunto de entrenamiento y conjunto de testing


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de variables


In [10]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Parte 2 - Construir la RNA




In [11]:
# Importar Keras y librerías adicionales
import keras
from keras.models import Sequential
from keras.layers import Dense

In [12]:
# Inicializar la RNA
classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))

# Añadir la segunda capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

# Añadir la capa de salida
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.4921 - accuracy: 0.7952
Epoch 2/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4290 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4238 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4202 - accuracy: 0.8154
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4172 - accuracy: 0.8255
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4152 - accuracy: 0.8290
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4134 - accuracy: 0.8305
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4121 - accuracy: 0.8317
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4111 - accuracy: 0.8305
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4100 - accura

## Parte 3 - Evaluar el modelo y calcular predicciones finales


### Predicción de los resultados con el Conjunto de Testing

In [13]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

### Predecir una nueva observación

Utiliza nuestro modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco:




*   Geografia: Francia
*   Puntaje de crédito: 600
*   Género masculino
*   Edad: 40 años de edad
*   Tenencia: 3 años.
*   Saldo: $ 60000

*   Número de productos: 2
*   ¿Este cliente tiene una tarjeta de crédito? Sí
*   ¿Es este cliente un miembro activo? Sí
*   Salario estimado: $ 50000

Entonces, ¿deberíamos decir adiós a ese cliente?

In [14]:
new_prediction = classifier.predict(sc_X.transform(np.array([[0,0,600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print(new_prediction)
print(new_prediction > 0.5)

[[0.09154963]]
[[False]]


### Elaborar una matriz de confusión



In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1545,   50],
       [ 268,  137]])

In [16]:
(cm[0][0]+cm[1][1])/cm.sum()

0.841